In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [11]:
# Scraper function using selenium

def get_jobs(keyword, num_jobs, verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="/Users/rohan/Documents/Studies/Python_Practice/DataScienceProjects/SalariesOfDataScientist/chromedriver", options=options)
    driver.set_window_size(1120, 1000)

    url = 'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=' +keyword+ ' &sc.keyword=' +keyword+ '&locT=&locId=&jobType='
    driver.get(url)
    jobs = []
    print("Number of jobs is ", num_jobs)
    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(4)
        print("---- In while loop ----")

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/div/div[2]/section/article/div[1]/ul/li[1]').click()
            
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_css_selector('[alt="Close"]').click()  #clicking to the X.
        except NoSuchElementException:
            pass

        
        #Going through each job in this page
        
        job_buttons = driver.find_elements_by_xpath('.//li[contains(@class,"react-job-listing")]')  #jl for Job Listing. These are the buttons we're going to click.
        
        for job_button in job_buttons:  
            print("Collecting jobs")
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(5)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    print("Collecting jobs")
                    company_name = driver.find_element_by_xpath('//div[@class="css-xuk5ye e1tk4kwz5"]').text
                    print("Company Name is {}".format(company_name))
                    location = driver.find_element_by_xpath('//div[@class="css-56kyx5 e1tk4kwz1"]').text
                    job_title = driver.find_element_by_xpath('//div[@class="css-1j389vi e1tk4kwz2"]').text
                    show_more = driver.find_element_by_xpath('//div[@class="css-t3xrds e856ufb2"]')
                   #job_description = driver.find_element_by_xpath('.//div[@id="jobDescriptionContainer desc"]').text
                    collected_successfully = True
                    print("No error so far")
                    
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath("(.//span[contains(@class,'css-1hbqxax e1wijj240')])").text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
                print("No Salary")             
           
                
            #Printing for debugging
            print("Value of verbose is {}".format(verbose))
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                #print("Job Description: {}".format(job_description[:500]))
                #print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            print("Before try block")
            try:
                print("Trying company tab")
                driver.find_element_by_xpath('//span[normalize-space()="Company"]').click()
                time.sleep(5)
                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    
                    
                    comp_attributes = driver.find_elements_by_xpath('(//div[@class="d-flex justify-content-start css-daag8o e1pvx6aw2"])')
                    counter = 0
                    attributes = {}
                    for element in comp_attributes:
                        # Each element is a property of company tab and its value. We add it in a key-value pair in dictionary
                        print(element.text)
                        attributes[element.text.split('\n')[0]] = element.text.split('\n')[1]
                    # Assigning value from dictionary to each variable
                
                    print(attributes.keys())
                    size = attributes.get("Size")
                    founded = attributes.get("Founded")
                    type_of_ownership = attributes.get("Type")
                    industry = attributes.get("Industry")
                    sector = attributes.get("Sector")
                    revenue = attributes.get("Revenue")
                    competitors = attributes.get("Competitors")      
                    
                    
                except NoSuchElementException: 
                    print("Company tab found but details missing")
                        
            
            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                print("Exception encountered")
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            
            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Company Name" : company_name,
            "Location" : location,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath("(//*[name()='svg'][@class='SVGInline-svg'])[3]").click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.


In [12]:
# Call to scraper function
df = get_jobs("data scientist",15, True)
df

Number of jobs is  15
---- In while loop ----
Progress: 0/15
Company Name is American Capital Group
4.3
No error so far
Value of verbose is True
Job Title: Data Scientist
Salary Estimate: Employer Provided Salary:$150K
Company Name: American Capital Group
4.3
Location: Bellevue, WA
Before try block
Trying company tab
Size
51 to 200 Employees
Founded
1986
Type
Company - Private
Industry
Real Estate
Sector
Real Estate
Revenue
Unknown / Non-Applicable
dict_keys(['Size', 'Founded', 'Type', 'Industry', 'Sector', 'Revenue'])
Progress: 1/15
Company Name is Northrop Grumman
4.0
No error so far
Value of verbose is True
Job Title: GBSD Principal Data Scientist – 5066
Salary Estimate: Employer Provided Salary:$96K - $144K
Company Name: Northrop Grumman
4.0
Location: Roy, UT
Before try block
Trying company tab
Size
10000+ Employees
Founded
1939
Type
Company - Public
Industry
Aerospace & Defense
Sector
Aerospace & Defense
Revenue
$10+ billion (USD)
dict_keys(['Size', 'Founded', 'Type', 'Industry', 

,Job Title,Salary Estimate,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Scientist,Employer Provided Salary:$150K,American Capital Group\n4.3,"Bellevue, WA",51 to 200 Employees,1986,Company - Private,Real Estate,Real Estate,Unknown / Non-Applicable,NaN
1,GBSD Principal Data Scientist – 5066,Employer Provided Salary:$96K - $144K,Northrop Grumman\n4.0,"Roy, UT",10000+ Employees,1939,Company - Public,Aerospace & Defense,Aerospace & Defense,$10+ billion (USD),NaN
2,Data Scientist,$92K - $149K (Glassdoor est.),Betacom\n3.6,"Bellevue, WA",201 to 500 Employees,1991,Company - Private,Telecommunications Services,Telecommunications,$100 to $500 million (USD),NaN
3,Data Scientist Consultant,Employer Provided Salary:$75 Per Hour,TBD Solutions LLC,"Grand Rapids, MI",-1,-1,-1,-1,-1,-1,-1.0
4,Data Scientist,Employer Provided Salary:$95K,AdVon Commerce,"Dublin, OH",-1,-1,-1,-1,-1,-1,-1.0
5,Data Scientist,Employer Provided Salary:$50K - $250K,Praescient Analytics\n3.4,"Washington, DC",51 to 200 Employees,2011,Company - Private,Consulting,Business Services,$10 to $25 million (USD),NaN
6,Data Scientist,$63K - $122K (Glassdoor est.),Cyient\n3.3,"Phoenix, AZ",10000+ Employees,1991,Company - Public,IT Services,Information Technology,$100 to $500 million (USD),NaN
7,Data Scientist - Remote,Employer Provided Salary:$115K - $150K,Emerald Resource Group,Remote,-1,-1,-1,-1,-1,-1,-1.0
8,Data Scientist,$56K - $98K (Glassdoor est.),Blain's Farm & Fleet\n3.3,"Janesville, WI",1001 to 5000 Employees,1955,Company - Private,Other Retail Stores,Retail,Unknown / Non-Applicable,NaN
9,"Data Scientist, Product",Employer Provided Salary:$130K - $140K,Ancera,"Atlanta, GA",-1,-1,-1,-1,-1,-1,-1.0


In [13]:
# Converting file to excel
df.to_excel('GlassdoorSalaries.xls')

<ipython-input-13-396e2c8b47df>:5: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel('test.xls')
